# <center>CFC Introductory Demo</center>

This demo includes a graphical interface to specify the module's properties and is meant as an introduction to it's functionality. The **Modulation**, **Sample rate** and **Frequency resolution** fields, translate directly to the equivalent module properties.

The **Number of samples** field is the length of the input signal. it populates a variable *n* that can be used in the **Signal** field. Any valid Python expression can be used for the input signal (defined by the *sig* variable). An additional cell has been provided with experimental signals that can be pasted into the Signal field. Note that the module uses a ring buffer of size $N$ to store the input samples. To avoid mixing unrelated input signals, a new instance should be initialized (which flushes the buffer) whenever a new input signal is chosen.

The output shows plots of the (uncorrected) input and (corrected) output signals. It also reports the detected offset.

A new instance should be created (using the **Init** button), whenever new parameters (or a new unrelated input signal) are specified. To clear the ouput, use the Jupyter Notebook's own shortcuts. Refer to the documentation in the Help menu.

In [ ]:
%matplotlib inline

import ipywidgets as widgets
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np

import sksdr
import utils

cfc = None

def init(b):
    global cfc, disp
    mod = modulation_widget.value
    sample_rate = sample_rate_widget.value
    freq_res = freq_res_widget.value
    cfc = sksdr.CoarseFrequencyComp(mod.order, sample_rate, freq_res)
    with disp:
        print('Initiated CFC with ' + repr(cfc))

def execute(b):
    global cfc, disp
    n = np.arange(num_samples_widget.value)
    _locals = {'n': n}
    exec(signal_widget.value, None, _locals)
    sig = _locals['sig']
    with disp:
        fig = plt.figure(figsize=(15,10))
        gs = gridspec.GridSpec(2, 2, figure=fig)
        sksdr.psd_plot(sig, cfc.sample_rate, 'Input Signal', fig=fig, gs=gs[0, 0])
        out, _, offset = cfc(sig)
        sksdr.psd_plot(out, cfc.sample_rate, 'Output Signal', fig=fig, gs=gs[0, 1])
        plt.show()
        print('Offset: ' + str(offset) + ' ' + 'Hz')

style = dict(utils.description_width_style)
settings_grid = widgets.GridspecLayout(2, 3)
settings_grid[0, 0] = modulation_widget = widgets.Dropdown(description='Modulation:', options=[('BPSK', sksdr.BPSK), ('QPSK', sksdr.QPSK)], value=sksdr.QPSK, continuous_update=False, style=style )
settings_grid[0, 1] = sample_rate_widget = widgets.BoundedFloatText(description='Sample rate (Hz):', value=1.0, min=0, max=np.finfo(float).max, continuous_update=False, style=style)
settings_grid[0, 2] = freq_res_widget = widgets.BoundedFloatText(description='Frequency resolution (Hz): ', value=0.000125, min=0, max=np.finfo(float).max, continuous_update=False, style=style)
settings_grid[1, 0] = num_samples_widget = widgets.BoundedIntText(description='Number of samples (n=):', value=100, min=1, max=np.iinfo(int).max, continuous_update=False, style=style)
settings_grid[1, 1:] = signal_widget = widgets.Textarea(description='Signal (sig=):', value='sig = np.exp(2j * np.pi / 12 * n)', continuous_update=False, style=style, layout=widgets.Layout(height='auto', width='auto'))
init_button = widgets.Button(description='Init', tooltip='Init')
init_button.on_click(init)
execute_button = widgets.Button(description='Execute', tooltip='Execute')
execute_button.on_click(execute)
disp = widgets.Output()
ui = widgets.VBox([
    settings_grid,
    widgets.HBox([init_button, execute_button]),
    disp
])
display(ui)

In [ ]:
# To test these examples, paste the code into the Signal field

# Simple complex exponencial with 12 samples per cyle
#sig = np.exp(2j * np.pi / 12 * n)

# 100 bits modulated with QPSK, upsampled by 4 and passed through a PhaseFrequencyOffset impairment. Sample_rate=100e3, freq_error=5e3
bits = np.random.randint(0, 2, 100)
psk = sksdr.PSKModulator(sksdr.QPSK, [0,1,3,2], 1, np.pi/4)
symbols = psk.modulate(bits)
fir = sksdr.FirInterpolator(4, sksdr.rrc(4, 0.5, 10))
_, frame = fir(symbols)
pfo = sksdr.PhaseFrequencyOffset(100e3, 5000, 0) 
sig, _ = pfo(frame)